# 📑📗This Python script make use of Outlook and Excel automation to generate Automated report and sending report to desired email address
# We are doing Excel Automation using Pandas module and Outlook Automation using pypiwin32 module

In [ ]:
#pip install pypiwin32 to work with windows operating sysytm
import win32com.client 
import datetime
import os
import pandas as pd
from datetime import date
import time
t1=time.time()
to_day = date.today()
print("Execution started")

# Creating an object for the outlook application.
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Creating an object to access Inbox of the outlook.
inbox=outlook.GetDefaultFolder(6)

#  We are extracting Excel attachments from Outlook for below given date
date='2022-04-25'
# Making folder where our files will be stored
fp=r"C:\Users\euakumn\OneDrive - Ericsson AB\Desktop\Outlook_attachment"+"\\"+date
os.mkdir(fp)
messages=inbox.Items
count=0
for i in messages:
   
   
    try:
        if i.SenderEmailAddress=='upm@xyz.com' and str(i.senton.date())==date and i.subject !="Hourly Trend of Data Payload":
            attachments=i.Attachments
            attachment = attachments.Item(1)
            # the name of attachment file      
            attachment_name = str(attachment)
        
            attachment.SaveASFile(r"C:\Users\euakumn\OneDrive - Ericsson AB\Desktop\Outlook_attachment"+"\\"+date+"\\"+attachment_name)
            print(i.SenderEmailAddress,i.senton.date())
            count+=1
  
        
    except AttributeError as e:
        pass
print(count,"files successfully extracted...!!!")    

fp=r"C:\Users\euakumn\OneDrive - Ericsson AB\Desktop\\Outlook_attachment"+"\\"+date

#  Now we are going to merge all the downloaded files from Outlook which we extracted using above code
print('Execution Merging started')
df1m=pd.DataFrame()
for path , subdir, files in os.walk(fp):    
     for name in files:
             with open(os.path.join(path,name), 'rb') as myfile:
                    print(name)
                    df1=pd.read_csv(myfile)
                    
                    df1.insert(0,'Circle',name)
                    
                    df1.columns.values[1] = "KPI"
                    
                    df1['Circle'] = df1['Circle'].str.split('_').str[1]
                    df1['Circle'] = df1['Circle'].str.split('_').str[0]
                    
                    df1m=df1m.append(df1,ignore_index=True,sort = False)

# Saving combined file                    
df1m.to_csv(r"C:\Users\euakumn\OneDrive - Ericsson AB\Desktop\Outlook_attachment"+"\\"+date+"\\"+'Mycom Executive Dashboard' +" "+ date+".csv",index = False)

fpq=r"C:\Users\euakumn\OneDrive - Ericsson AB\Desktop\\Outlook_attachment"+"\\"+date+"\\"+'Mycom Executive Dashboard' +" "+ date+".csv"

# Reading Master_maping file from where we need to VLOOKUP for some columns in Combined file
fm=r"C:\Users\euakumn\OneDrive - Ericsson AB\Desktop\Master_maping.xlsx"
dfm=pd.read_excel(fm,sheet_name='Mycom Executive Dashboard 2021-')

df=pd.read_csv(fpq)

xy=pd.merge(df,dfm,on=['Circle','KPI'])
k=['Tech','Layer','Mapped KPI']   
y=2
for i in k:
    
    w= xy[i]
    xy = xy.drop(columns=[i])
    xy.insert(loc=y, column=i, value=w)
    y=y+1
    
# Again saving Final Mapped Dashboard after some VLOOKUP from Master_Mapping file    
xy.to_excel(r"C:\Users\euakumn\OneDrive - Ericsson AB\Desktop\Outlook_attachment"+"\\"+date+"\\"+'Mapped_Mycom Executive Dashboard' +" "+ date+".xlsx",index = False)


# Now our task is to send the Mapped_Mycom Executive dashboard to desired Email address
outlook = win32com.client.Dispatch('Outlook.Application')
message = outlook.CreateItem(0)


message.To = 'abcd@gamil.com'
message.CC = 'pqrs@gmail.com'


message.Subject = f'Mycomm Quality Executive Dashboard {date}'
html_body = """
    <div>
        <h1 style="font-family: 'Lucida Sans'; font-size: 15; color:#000000;">
            Hi sir, 
        </h1>
        <span style="font-family: 'Lucida Sans'; font-size: 15; color: ##000000;">
            Pls find  {}!!
            
            
            
        </span>
      <br>
      <br>
      <br>
      <br>
      <br>
      <br>
      
    </div><br>
    <div>
        <img src="https://internal.ericsson.com/sites/default/files/styles/media_entity_ckeditor_max/public/Compass_0.jpg?itok=E47l88GD" width=10%>
    </div>
   <span style="font-family: 'Lucida Sans'; font-size: 15; color: #000000;">
            <p>Thanks and regard!!
            <br>Anuj Kumar
            </p>
            
            
        </span>
    """

message.HTMLBody = html_body.format(message.subject)
# Adding attachments in Our Outlook mail
message.Attachments.Add(r"C:\Users\euakumn\OneDrive - Ericsson AB\Desktop\Outlook_attachment"+"\\"+date+"\\"+'Mapped_Mycom Executive Dashboard' +" "+ date+".xlsx")
message.save()
print("Email sent Successfully with attachment")
t2=time.time()
print(f"Completed the execution in {t2-t1} seconds")
